# [LangChain Quickstart Guide](https://python.langchain.com/docs/get_started/quickstart)
Derived from the LangChain link above.

Load the environmental variables form .env file

In [ ]:
import json
import textwrap
from dotenv import load_dotenv
import os
load_dotenv()

We need to first: `pip install langchain`

In [ ]:
def printlongtext(text, max_lines=10):
    short_strings = textwrap.wrap(text, 80)
    lines = len(short_strings)
    if lines < max_lines:
        max_lines = lines
    for index in range(0, max_lines):
        print(short_strings[index])
        

## LLM Chain
To use the OpenAI models, we first need to:</br> 
>`pip install langchain-openai`<br/>
>Make sure openai credentials are in the .env file.
#### Pick one of the following cells to define the llm.

In [ ]:
# OpenAI LLMs
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    # model="gpt-4",
    model="gpt-3.5-turbo",
    temperature=0,
)

In [ ]:
# Ollama LLMs
from langchain_community.llms import Ollama
llm = Ollama(
    # model = "llama2"
    model = "gemma"
)

Invoke the llm picked from above:

In [ ]:
result =llm.invoke("what's the difference between langchain and llamaindex?")

Examine the result

In [ ]:
# print(result.content)
print(result)
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
# print(json.dumps(result.json(), indent=4))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

Combine prompt and llm into a chain.

In [ ]:
chain = prompt | llm 

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
result = chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
# result is no longer an json object, it is a string
print(result)

## Retrieval Chain

WebBaseLoader uses beautifulsoup to scrape text from a web page.<br/>
We first need to: `pip install beautifulsoup4`

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [ ]:
printlongtext(docs[0].page_content.strip("\n"), 20)
print("-" * 80)
print(json.dumps(docs[0].metadata, indent=4))

#### This time we will use a local embedding llama2 model from ollama.
First we need to install ollama.<br/>
Then we need to pull the model we need to use: <br/>
`ollama pull <model_name>`

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

Need to do this first: `pip install faiss-cpu`

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [ ]:
printlongtext(response["answer"])

## Conversation Retrieval Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
printlongtext(result[0].page_content)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
printlongtext(result["answer"])

## Agent

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

Need Tavily API Key for this. (Check .env file)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [ ]:
tools = [retriever_tool, search]

First need to: `pip install langchainhub`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
result = agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
print(json.dumps(result, indent=4))
print("-" * 80)
printlongtext(result["output"], 20)

In [ ]:
sf_weather = agent_executor.invoke({"input": "what is the weather in SF?"})

In [ ]:
printlongtext(sf_weather["output"], 20)

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result_ls = agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
printlongtext(result_ls["output"], 20)

## Serving with LangServe

Refer to serve.py

Also needs to: `pip install "langserve[all]"`

# LangServe client

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8080/agent/")
result =remote_chain.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []  # Providing an empty list as this is the first call
})

In [ ]:
print(result["output"])